In [ ]:
#pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import pandas as pd
import time

In [2]:
# 初始化 WebDriver
driver = webdriver.Chrome()

# 目标 URL
url = 'https://tj.esf.fang.com/house-a052/'

# 打开网页
driver.get(url)

# 存储所有页面的数据
string_list = []

# 最大爬取页数
MAX_PAGES = 20

# 定义数据提取函数
def extract_house_data(row):
    """从单行数据中提取房源信息"""
    try:
        title = row.find_element(By.CSS_SELECTOR, "h4 a").text
        details_text = row.find_element(By.CSS_SELECTOR, "p.tel_shop").text
        details = details_text.split("|")
        house_type = details[0].strip() if len(details) > 0 else None
        area = details[1].strip() if len(details) > 1 else None
        floor = details[2].strip() if len(details) > 2 else None
        direction = details[3].strip() if len(details) > 3 else None
        built_year = details[4].strip() if len(details) > 4 else None
        name = row.find_element(By.CSS_SELECTOR, "p.add_shop a").text
        locate = row.find_element(By.CSS_SELECTOR, "p.add_shop span").text
        total_price = row.find_element(By.CSS_SELECTOR, "span.red b").text
        price_dd = row.find_element(By.CSS_SELECTOR, "dd.price_right")
        total_price = price_dd.find_element(By.CSS_SELECTOR, "span.red").text
        price_per_m2 = price_dd.find_element(By.CSS_SELECTOR, "span:not(.red)").text
        return [title, house_type, area, floor, direction, built_year, name, locate, total_price, price_per_m2]
    except NoSuchElementException:
        return None  # 如果某一行数据缺失，返回 None

# 主循环
for i in range(MAX_PAGES):
    try:
        # 等待表格加载完成
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'shop_list'))
        )
        rows = table.find_elements(By.TAG_NAME, 'dl')
        data = []

        # 提取每一行数据
        for row in rows:
            house_data = extract_house_data(row)
            if house_data:
                data.append(house_data)

        # 将当前页数据存储到 DataFrame
        df = pd.DataFrame(data, columns=[
            '标题', '户型', '面积', '层型', '朝向', '建成时间', '小区名', '区域', '总价', '单价'
        ])
        string_list.append(df)
        print(f"已爬取第 {i + 1} 页数据")

        # 翻页
        next_page = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'last'))
        )
        next_page.click()
        time.sleep(2)  # 等待页面加载
    except TimeoutException:
        print("已到达最后一页或页面加载超时")
        break
    except StaleElementReferenceException:
        print("页面元素已过期，重新尝试")
        continue
    except Exception as e:
        print(f"发生异常: {e}")
        break

# 关闭浏览器
driver.quit()



已爬取第 1 页数据
已爬取第 2 页数据
已爬取第 3 页数据
已爬取第 4 页数据
已爬取第 5 页数据
已爬取第 6 页数据
已爬取第 7 页数据
已爬取第 8 页数据
已爬取第 9 页数据
已爬取第 10 页数据
已爬取第 11 页数据
已爬取第 12 页数据
已爬取第 13 页数据
已爬取第 14 页数据
已爬取第 15 页数据
已爬取第 16 页数据
已爬取第 17 页数据
已爬取第 18 页数据
已爬取第 19 页数据
已爬取第 20 页数据


In [3]:
# 合并所有数据
if string_list:
    df_full = pd.concat(string_list, ignore_index=True)
    df_full.to_csv('esf_wuqing.csv', index=False, encoding='utf_8_sig')  # 保存为 CSV 文件
    print("数据已保存")

    # 数据描述
    print(df_full.describe())
else:
    print("未爬取到任何数据")

数据已保存
                                标题    房型    面积        层型    朝向    建成时间   小区名  \
count                         1200  1200  1200      1200  1200    1200  1200   
unique                         944    15   688       107    27      43   186   
top     希望源于失望,奋斗始于忧患适合的房子在于一个懂你的!  3室2厅  109㎡  中层 （共7层）   南北向  2006年建  枫丹天城   
freq                             4   521    13        65   704     430    59   

                          区域    总价        单价  
count                   1200  1200      1200  
unique                   149   223       922  
top     武清-新城雍阳西道与翠亨路交汇路口西北侧  140万  10000元/㎡  
freq                      59    23         5  
